#### 모델 평가 지표

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


##### - 데이터 전처리

###### 1. Feature selection

In [4]:
df = df[['survived','pclass','sex','age','sibsp','parch','fare','embarked','deck']]
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,deck
0,0,3,male,22.0,1,0,7.2500,S,NaN
1,1,1,female,38.0,1,0,71.2833,C,C
2,1,3,female,26.0,0,0,7.9250,S,NaN
3,1,1,female,35.0,1,0,53.1000,S,C
4,0,3,male,35.0,0,0,8.0500,S,NaN


In [5]:
df.isna().sum()

survived      0
pclass        0
sex           0
age         177
sibsp         0
parch         0
fare          0
embarked      2
deck        688
dtype: int64

###### 2. 결측치 처리

In [8]:
# age - 평균값으로 대체
df.age.fillna(df['age'].mean(), inplace=True)
df.age.isnull().sum()

0

In [9]:
# embarked - 최빈값으로 대체
df.embarked.value_counts()

S    644
C    168
Q     77
Name: embarked, dtype: int64

In [10]:
df.embarked.fillna('S', inplace=True)
df.embarked.isnull().sum()

0

In [11]:
df.drop(columns=['deck'], inplace=True)
df.isna().sum().sum()

0

###### 3. 카테고리 데이터 처리

In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [14]:
df.sex = le.fit_transform(df.sex)
df.embarked = le.fit_transform(df.embarked)
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


###### 4. Train/Test로 분할

In [15]:
X = df.iloc[:,1:].values
y = df['survived'].values

In [20]:
# y값 분포
np.unique(y, return_counts=True)

(array([0, 1], dtype=int64), array([549, 342], dtype=int64))

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=2022
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 7), (179, 7), (712,), (179,))

In [22]:
np.unique(y_train, return_counts=True)

(array([0, 1], dtype=int64), array([439, 273], dtype=int64))

###### 5. RandomForest 모델 학습

In [23]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=2022)
rfc.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 2022,
 'verbose': 0,
 'warm_start': False}

In [24]:
rfc.fit(X_train,y_train)

RandomForestClassifier(random_state=2022)

###### 6. RandomForest 모델 평가

In [26]:
rfc.score(X_test,y_test)

0.8324022346368715

###### 7. GridSearch 수행

In [27]:
params = {
    'max_depth':[2,4,6,8],
    'min_samples_split':[2,4,6]
}

In [29]:
from sklearn.model_selection import GridSearchCV
grid_rf = GridSearchCV(rfc, params, scoring='accuracy', cv=5)
grid_rf.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=2022),
             param_grid={'max_depth': [2, 4, 6, 8],
                         'min_samples_split': [2, 4, 6]},
             scoring='accuracy')

In [30]:
grid_rf.best_params_

{'max_depth': 4, 'min_samples_split': 4}

In [31]:
params = {
    'max_depth':[3,4,5],
    'min_samples_split':[3,4,5]
}
grid_rf = GridSearchCV(rfc, params, scoring='accuracy', cv=5)
%time grid_rf.fit(X_train,y_train)


CPU times: total: 4.31 s
Wall time: 4.32 s


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=2022),
             param_grid={'max_depth': [3, 4, 5],
                         'min_samples_split': [3, 4, 5]},
             scoring='accuracy')

In [32]:
grid_rf.best_params_

{'max_depth': 4, 'min_samples_split': 3}

In [33]:
best_rf = grid_rf.best_estimator_
best_rf.score(X_test,y_test)

0.8212290502793296

In [47]:
pred_rf = best_rf.predict(X_test)

###### 7. 테스트 데이터 하나 수행

In [34]:
X_test[25], y_test[25]

(array([ 3.  ,  1.  , 45.  ,  0.  ,  0.  ,  8.05,  2.  ]), 1)

In [35]:
best_rf.predict(X_test[25].reshape(1,-1))

array([0], dtype=int64)

In [40]:
df.groupby(['sex'])['survived'].mean()

sex
0    0.742038
1    0.188908
Name: survived, dtype: float64

###### 8. 엉터리 분류기

In [49]:
from sklearn.base import BaseEstimator

class MyClassifier(BaseEstimator):
    # fit(), predict(), method만 재정의(Overriding)
    def fit(self, X, y):
        pass
    def predict(self, X):
        pred = np.zeros(X.shape[0], int) # X 행의 갯수만큼 0으로 초기 배열 생성
        for i in range(X.shape[0]):
            # 여성이면 생존으로 처리
            if X[i,1] == 0:
                pred[i] = 1
        return pred

In [50]:
my_clf = MyClassifier()
my_clf.fit(X_train,y_train)
pred_my = my_clf.predict(X_test)

In [51]:
X_test[8,1], pred_my[8]

(0.0, 1)

In [52]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred_my)

0.7877094972067039

In [56]:
sdf = pd.DataFrame({'y':y_test,'rf':pred_rf,'my':pred_my})
sdf

,y,rf,my
0,1,0,0
1,0,0,0
2,1,0,1
3,0,0,0
4,0,0,0
...,...,...,...
174,0,0,0
175,1,1,1
176,0,0,0
177,1,1,1


In [46]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred_my)

array([[96, 14],
       [24, 45]], dtype=int64)

In [57]:
confusion_matrix(y_test,pred_rf)

array([[103,   7],
       [ 25,  44]], dtype=int64)

In [58]:
from sklearn.metrics import precision_score, recall_score

In [60]:
# 정밀도(Precision Score) - TP / (FP + TP)
precision_score(y_test, pred_rf), precision_score(y_test, pred_my)

(0.8627450980392157, 0.7627118644067796)

In [61]:
# 재현율(Recall Score) - TP / (FN + TP)
recall_score(y_test, pred_rf), recall_score(y_test, pred_my)

(0.6376811594202898, 0.6521739130434783)

In [62]:
# f1 스코어(F1 Score)
from sklearn.metrics import f1_score
f1_score(y_test, pred_rf), f1_score(y_test,pred_my)

(0.7333333333333333, 0.703125)

In [63]:
# AUC Score
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, pred_rf), roc_auc_score(y_test, pred_my)

(0.787022397891963, 0.7624505928853755)